# A tour of PyCIEMSS interfaces and functionality

### Load dependencies and interfaces

In [1]:
import os
import pyciemss
from pyciemss.interfaces import calibrate

### Select models and data

In [2]:
MODEL_PATH = "https://raw.githubusercontent.com/DARPA-ASKEM/simulation-integration/main/data/models/"
DATA_PATH = "https://raw.githubusercontent.com/DARPA-ASKEM/simulation-integration/main/data/datasets/"

model1 = os.path.join(MODEL_PATH, "SEIRHD_NPI_Type1_petrinet.json")
model2 = os.path.join(MODEL_PATH, "SEIRHD_NPI_Type2_petrinet.json")

dataset1 = os.path.join(DATA_PATH, "traditional.csv")

### Set parameters for sampling

In [3]:
start_time = 0.0
end_time = 100.
logging_step_size = 10.0
num_samples = 3

## Sample interface
Take `num_samples` number of samples from the (prior) distribution invoked by the chosen model.

### Sample from model 1

In [4]:
result1 = pyciemss.sample(model1, end_time, logging_step_size, num_samples, start_time=start_time)
result1["unprocessed_result"]

{'persistent_beta_c': tensor([0.6242, 0.1793, 0.5979]),
 'persistent_kappa': tensor([0.3027, 0.0840, 0.5343]),
 'persistent_gamma': tensor([0.4600, 0.1461, 0.3017]),
 'persistent_hosp': tensor([0.1435, 0.0779, 0.1470]),
 'persistent_death_hosp': tensor([0.0420, 0.0260, 0.0205]),
 'persistent_I0': tensor([ 8.6543, 14.3962, 13.7999]),
 'D_state': tensor([[2.1160e-01, 4.7950e-01, 6.4792e-01, 7.4287e-01, 7.9512e-01, 8.2370e-01,
          8.3931e-01, 8.4784e-01, 8.5249e-01],
         [4.3300e-02, 1.1175e-01, 1.6321e-01, 1.9738e-01, 2.1950e-01, 2.3375e-01,
          2.4291e-01, 2.4881e-01, 2.5260e-01],
         [1.4265e-01, 6.3840e-01, 1.8854e+00, 4.9729e+00, 1.2609e+01, 3.1485e+01,
          7.8092e+01, 1.9283e+02, 4.7327e+02]]),
 'E_state': tensor([[1.7254e+01, 9.4135e+00, 5.1396e+00, 2.8061e+00, 1.5321e+00, 8.3647e-01,
          4.5669e-01, 2.4934e-01, 1.2731e-01],
         [1.1723e+01, 7.1257e+00, 4.5712e+00, 2.9400e+00, 1.8911e+00, 1.2164e+00,
          7.8242e-01, 5.0328e-01, 2.7477e-0

In [5]:
result1['data'].head()

,timepoint_id,sample_id,persistent_beta_c_param,persistent_kappa_param,persistent_gamma_param,persistent_hosp_param,persistent_death_hosp_param,persistent_I0_param,D_state_state,E_state_state,H_state_state,I_state_state,R_state_state,S_state_state,infected_observable_state,exposed_observable_state,hospitalized_observable_state,dead_observable_state
0,0,0,0.624231,0.302744,0.459952,0.143516,0.042003,8.654272,0.211603,17.254230,3.622669,10.774392,56.511272,19339972.0,10.774392,17.254230,3.622669,0.211603
1,1,0,0.624231,0.302744,0.459952,0.143516,0.042003,8.654272,0.479496,9.413510,2.587079,5.891737,94.504311,19339934.0,5.891737,9.413510,2.587079,0.479496
2,2,0,0.624231,0.302744,0.459952,0.143516,0.042003,8.654272,0.647915,5.139568,1.495113,3.216777,115.758553,19339922.0,3.216777,5.139568,1.495113,0.647915
3,3,0,0.624231,0.302744,0.459952,0.143516,0.042003,8.654272,0.742869,2.806087,0.827481,1.756289,127.431374,19339912.0,1.756289,2.806087,0.827481,0.742869
4,4,0,0.624231,0.302744,0.459952,0.143516,0.042003,8.654272,0.795118,1.532063,0.453300,0.958892,133.813751,19339910.0,0.958892,1.532063,0.453300,0.795118


### Sample from model 2

In [6]:
result2 = pyciemss.sample(model2, end_time, logging_step_size, num_samples, start_time=start_time)
result2['data'].head()

,timepoint_id,sample_id,persistent_beta_c_param,persistent_beta_nc_param,persistent_kappa_param,persistent_gamma_param,persistent_hosp_param,persistent_death_hosp_param,persistent_I0_param,D_state_state,E_state_state,H_state_state,I_state_state,R_state_state,S_state_state,infected_observable_state,exposed_observable_state,hospitalized_observable_state,dead_observable_state
0,0,0,0.160398,0.508773,0.514139,0.441564,0.064289,0.080559,9.252161,0.222313,40.670559,2.398664,21.768782,77.614960,19339896.0,21.768782,40.670559,2.398664,0.222313
1,1,0,0.160398,0.508773,0.514139,0.441564,0.064289,0.080559,9.252161,0.701271,52.374638,3.486899,28.045502,185.455078,19339768.0,28.045502,52.374638,3.486899,0.701271
2,2,0,0.160398,0.508773,0.514139,0.441564,0.064289,0.080559,9.252161,1.345864,67.456810,4.544889,36.121891,324.671875,19339578.0,36.121891,67.456810,4.544889,1.345864
3,3,0,0.160398,0.508773,0.514139,0.441564,0.064289,0.080559,9.252161,2.179833,86.880646,5.860922,46.523067,504.019257,19339394.0,46.523067,86.880646,5.860922,2.179833
4,4,0,0.160398,0.508773,0.514139,0.441564,0.064289,0.080559,9.252161,3.254447,111.895035,7.549476,59.918148,735.013794,19339120.0,59.918148,111.895035,7.549476,3.254447


## Ensemble Sample Interface
Sample from an ensemble of model 1 and model 2 

In [7]:
model_paths = [model1, model2]
solution_mappings = [lambda x : x, lambda x : x] # Conveniently, these two models operate on exactly the same state space, with the same names.

ensemble_result = pyciemss.ensemble_sample(model_paths, solution_mappings, end_time, logging_step_size, num_samples, start_time=start_time)
ensemble_result['data'].head()

,timepoint_id,sample_id,model_0/persistent_beta_c_param,model_0/persistent_kappa_param,model_0/persistent_gamma_param,model_0/persistent_hosp_param,model_0/persistent_death_hosp_param,model_0/persistent_I0_param,model_1/persistent_beta_c_param,model_1/persistent_beta_nc_param,...,model_0/H_state_state,model_0/I_state_state,model_0/R_state_state,model_0/S_state_state,model_1/D_state_state,model_1/E_state_state,model_1/H_state_state,model_1/I_state_state,model_1/R_state_state,model_1/S_state_state
0,0,0,0.415438,0.322769,0.495628,0.153126,0.048887,12.027315,0.684033,0.397891,...,4.180867,10.492547,62.530724,19339948.0,0.113527,43.070324,2.173762,43.590801,49.690903,19339872.0
1,1,0,0.415438,0.322769,0.495628,0.153126,0.048887,12.027315,0.684033,0.397891,...,2.871795,5.573850,102.036423,19339924.0,0.468467,83.698334,4.708426,85.152145,157.981125,19339704.0
2,2,0,0.415438,0.322769,0.495628,0.153126,0.048887,12.027315,0.684033,0.397891,...,1.612748,2.958564,123.545570,19339932.0,1.182108,163.121994,9.241726,165.959930,369.491272,19339326.0
3,3,0,0.415438,0.322769,0.495628,0.153126,0.048887,12.027315,0.684033,0.397891,...,0.868001,1.570385,135.035248,19339904.0,2.575937,317.902039,18.020260,323.439484,781.764343,19338592.0
4,4,0,0.415438,0.322769,0.495628,0.153126,0.048887,12.027315,0.684033,0.397891,...,0.462348,0.833549,141.143768,19339900.0,5.292745,619.494507,35.119476,630.308716,1585.224243,19337162.0


## Calibrate interface
Calibrate a model to a dataset by mapping model state varibale or observables to columns in the dataset

In [8]:
data_mapping = {"Infected": "I"} # data_mapping = "column_name": "observable/state_variable"
num_iterations = 10
calibrated_results = calibrate(model1, dataset1, data_mapping=data_mapping, num_iterations=num_iterations)
parameter_estimates = calibrated_results["inferred_parameters"]
calibrated_results

{'inferred_parameters': AutoGuideList(
   (0): AutoDelta()
   (1): AutoLowRankMultivariateNormal()
 ),
 'loss': 242.64169976115227}

In [9]:
parameter_estimates

AutoGuideList(
  (0): AutoDelta()
  (1): AutoLowRankMultivariateNormal()
)

In [10]:
point_estimates = parameter_estimates(0)
point_estimates

{'persistent_beta_c': tensor(0.4251, grad_fn=<ExpandBackward0>),
 'persistent_kappa': tensor(0.4595, grad_fn=<ExpandBackward0>),
 'persistent_gamma': tensor(0.2547, grad_fn=<ExpandBackward0>),
 'persistent_hosp': tensor(0.0955, grad_fn=<ExpandBackward0>),
 'persistent_death_hosp': tensor(0.0643, grad_fn=<ExpandBackward0>),
 'persistent_I0': tensor(5.9287, grad_fn=<ExpandBackward0>)}

In [11]:
gaussian_estimates = parameter_estimates(1)
gaussian_estimates

{'persistent_beta_c': tensor(0.4529, grad_fn=<ExpandBackward0>),
 'persistent_kappa': tensor(0.4055, grad_fn=<ExpandBackward0>),
 'persistent_gamma': tensor(0.2456, grad_fn=<ExpandBackward0>),
 'persistent_hosp': tensor(0.1095, grad_fn=<ExpandBackward0>),
 'persistent_death_hosp': tensor(0.0564, grad_fn=<ExpandBackward0>),
 'persistent_I0': tensor(6.6700, grad_fn=<ExpandBackward0>)}

## Pass the parameter estimates to `sample` to sample from the calibrated model

In [12]:
calibrated_sample_results = pyciemss.sample(model1, end_time, logging_step_size, num_samples, 
                start_time=start_time, inferred_parameters=parameter_estimates)
calibrated_sample_results

{'data':     timepoint_id  sample_id  persistent_beta_c_param  persistent_kappa_param  \
 0              0          0                 0.453773                0.412738   
 1              1          0                 0.453773                0.412738   
 2              2          0                 0.453773                0.412738   
 3              3          0                 0.453773                0.412738   
 4              4          0                 0.453773                0.412738   
 5              5          0                 0.453773                0.412738   
 6              6          0                 0.453773                0.412738   
 7              7          0                 0.453773                0.412738   
 8              8          0                 0.453773                0.412738   
 9              0          1                 0.455092                0.442731   
 10             1          1                 0.455092                0.442731   
 11             2   

In [13]:
# TODO:
# - Add examples for calibrate_ensemble and optimize interfaces as they become available
# - Plot results